In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "1"
import keras
from tensorflow.keras.datasets import cifar100, fashion_mnist,mnist, cifar10
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers,callbacks
#GPU
gpu=1
import gc
import tensorflow as tf
counter_s=0
gc.collect()
import os
from tensorflow.keras.regularizers import l2
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))

In [ ]:
from tensorflow.keras import layers
from plexusnet.architecture import PlexusNet

In [3]:
keras.__version__, tf.__version__

('2.5.0', '2.5.0')

In [4]:
import keras_tuner as kt

In [5]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()


In [6]:
y_train=to_categorical(y_train, 10)
X_train=X_train/255.0

In [7]:
def build_model(hp):
    depth=hp.Int('depth', min_value=3,max_value=4)
    length=hp.Int('length',min_value=2 ,max_value=6)
    junction=hp.Int('junction', min_value=1,max_value=6)
    initial_filter=hp.Int('initial_filter', min_value=4, max_value=16, step=4)
    type_of_block=hp.Choice('type_of_block', ['inception', 'resnet'])
    WEIGHT_DECAY=hp.Float("WEIGHT_DECAY",min_value=1e-6, max_value=1e-3, sampling="reverse_log")
    
    GlobalPooling=hp.Choice("GlobalPooling",["avg","max"])
    model=PlexusNet(depth=depth,length=length,junction=junction,n_class=10, initial_filter=initial_filter,
                    input_shape=(32,32), kernel_regularizer=l2(WEIGHT_DECAY), type_of_block=type_of_block, GlobalPooling=GlobalPooling, run_255_division=False,escape_fc1=True).model
    # create model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



# log-uniform: understand as search over p = exp(x) by varying x
tuner = kt.Hyperband(
    hypermodel=build_model,
    objective="val_accuracy",
    max_epochs=1,
    factor=3,
    hyperband_iterations=5,
    directory="plexus_netresults_dir",
    project_name="mnist",
    overwrite=True,
)

In [ ]:
x_val= X_train[45000:]
y_val= y_train[45000:]
x_train= X_train[:45000]
y_train= y_train[:45000]
tuner.search(x_train, y_train, epochs=1, validation_data=(x_val, y_val))

In [ ]:
tuner.results_summary()